In [125]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
import datetime
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

pd.options.mode.chained_assignment = None  # default='warn'


# Load the dataset
#file_path = "__sprint_1_alt_no_mans_sky_labelled_steam_review_data.csv"
file_path = "datasets/__sprint2_no_mans_sky_steam_review_data_split_reviews_remergeable_FINAL_LABELS.csv"
df = pd.read_csv(file_path, encoding='latin1', low_memory=False)

#heuristic
#informative
#df.head(10)

wip_df = df.drop(columns=[#'recommendationid'
                          #,'clean_sentence'
                          #,'tokenized_sentence'
                          #,'voted_up'
                          #'language'
                          #,'timestamp_created'
                          #,'timestamp_updated'
                          #,'votes_up'
                          #,'votes_funny'
                          #,'weighted_vote_score'
                          #,'steam_purchase'
                          #,'author_num_games_owned'
                          #,'author_num_reviews'
                          #,'author_playtime_forever'
                          #,'author_last_played'
                          #,'informative'
                          #,'heuristic'
                 ])
#wip_df.head(10)
informative_df = wip_df.drop(columns=['recommendationid', 'tokenized_sentence', 'voted_up', 'heuristic'])
heuristic_df = wip_df.drop(columns=['recommendationid', 'tokenized_sentence', 'voted_up','informative'])
#informative_df.head(10)
#heuristic_df.head(10)
df.head(10)

,recommendationid,clean_sentence,tokenized_sentence,voted_up,order,informative,heuristic
0,24845425,yeah gr8 db8 ir8 st8 hey apreci8 gr8 wan mayb ...,yeah m8 this is gr8 no db8 i r8 it an 8 i h8 2...,1,0,NaN,NaN
1,24845425,rel8 str8 appreci8 congratul8,i rel8 str8 appreci8 and congratul8,1,1,NaN,NaN
2,24845425,NaN,i r8 this b8 an 88,1,2,NaN,NaN
3,24845425,plz str8 ir8,plz no h8 im str8 ir8,1,3,NaN,NaN
4,24845425,cre8 cant,cre8 more cant w8,1,4,NaN,NaN
5,24845425,convers8 wont ber8 number 8888888 ask,we should convers8 i wont ber8 my number is 88...,1,5,NaN,NaN
6,24845425,calls st8,no calls l8 or out of st8,1,6,NaN,NaN
7,24845425,ask loc8,if on a d8 ask k8 to loc8,1,7,NaN,NaN
8,24845425,even full pl8 always time communic8 dont hesit8,even with a full pl8 i always have time to com...,1,8,NaN,NaN
9,24845428,pretty good tbh edit pretty good fuck sean lyi...,its pretty good tbh edit its pretty good but f...,0,0,NaN,NaN


In [127]:
wip_df.columns

Index(['recommendationid', 'clean_sentence', 'tokenized_sentence', 'voted_up',
       'order', 'informative', 'heuristic'],
      dtype='object')

In [135]:
informative_df.drop_duplicates()['informative'].value_counts()

informative
0.0    2625
1.0    1581
Name: count, dtype: int64

In [229]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

#Playability informative Naives Bayes Classification

X = informative_df.drop_duplicates()[(informative_df['informative'].notnull()) & (informative_df['informative'] != '')]
X = X.dropna()
X = (X.groupby('informative', group_keys=False)
         .apply(lambda g: g.sample(n=min(1500, len(g)), random_state=5))
      )
X['informative'].value_counts()

informative
0.0    1500
1.0    1500
Name: count, dtype: int64

In [231]:


y = X['informative']
X = X.drop(columns=['informative'])
#print(X.count())
#print(y.count())

# Convert the text data to a bag-of-words representation
vectorizer = CountVectorizer()
texts = X['clean_sentence']
X = vectorizer.fit_transform(texts)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=5, stratify=y)

# Initialize the Naive Bayes classifier (MultinomialNB is well-suited for count data)
classifier = MultinomialNB()

# Train the classifier on the training data
classifier.fit(X_train, y_train)

# Predict the labels for the test set
y_pred = classifier.predict(X_test)

# Evaluate the classifier performance
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

Accuracy: 0.8133333333333334
Classification Report:
               precision    recall  f1-score   support

         0.0       0.84      0.78      0.81       450
         1.0       0.79      0.85      0.82       450

    accuracy                           0.81       900
   macro avg       0.82      0.81      0.81       900
weighted avg       0.82      0.81      0.81       900



In [235]:
print(informative_df['informative'].value_counts())
print(informative_df['clean_sentence'].count())

informative
0.0    8099
1.0    1596
Name: count, dtype: int64
515471


In [237]:
#Execution started at 16:37

In [239]:
print(datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S"))

20.Mar 2025 23:55:48


In [241]:
print(informative_df['informative'].value_counts())

for row in informative_df[(informative_df['informative'].isnull()) | (informative_df['informative'] == '') ].itertuples(index=True, name='Row'):
    #print(row)
    #if len(row.clean_sentence):
    vect = vectorizer.transform([str(row.clean_sentence)])
    informative_df.loc[row.Index, 'informative'] = classifier.predict(vect)
    #else:
    #    informative_df.loc[row.Index, 'informative'] = 0

for row in wip_df[(wip_df['informative'].isnull()) | (wip_df['informative'] == '') ].itertuples(index=True, name='Row'):
    vect = vectorizer.transform([str(row.clean_sentence)])
    wip_df.loc[row.Index, 'informative'] = classifier.predict(vect)

print(informative_df['informative'].value_counts())

informative
0.0    8099
1.0    1596
Name: count, dtype: int64
informative
0.0    274687
1.0    248961
Name: count, dtype: int64


In [242]:
print(datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S"))

21.Mar 2025 00:01:21


In [243]:
#informative_df[(informative_df['informative'] == 0)]['clean_sentence'].tail(50)

In [246]:
print(datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S"))

21.Mar 2025 00:02:48


In [324]:
heuristic_df.loc[(heuristic_df['heuristic'].notnull())].head(10)

,clean_sentence,order,heuristic
494,even tho kind added 219 support still stretche...,2,F
593,sit loading screen minutes crashes,0,F
599,games fantastic upon launching screen flashes ...,0,F
768,lets talk menus bad port obviously frustrating...,8,U
887,crashes loading space scene,0,F
892,terrible fps terrible graphic quality loading ...,0,F
979,stuttering freezing fixed things like searchin...,5,F
998,lowest fps ever stuttering everywhere,1,F
1165,issues launching game,3,F
1433,deafult 30fps unintuative menu system initally...,1,U


In [338]:
print(heuristic_df.head(20))
# 'heuristic'
heuristic_training = heuristic_df.loc[(heuristic_df['heuristic'].notnull()) & (heuristic_df['heuristic'] != '')]
heuristic_training = heuristic_df.loc[((heuristic_df['heuristic'] =='G')|(heuristic_df['heuristic'] =='U')|(heuristic_df['heuristic'] =='F')) & (heuristic_df['heuristic'] != '')]
#heuristic_training = heuristic_training.drop_duplicates()

heuristic_training = heuristic_training.drop_duplicates()

#Tidy up the label values
for row in heuristic_training.itertuples(index=True, name='Row'):
    heuristic_training.loc[row.Index, 'heuristic'] = str(heuristic_training.loc[row.Index, 'heuristic']).replace('"','').replace("'",'').replace(' ','').replace('[','').replace(']','').split(',')



heuristic_training.dropna(how='any')
heuristic_training.describe()

#heuristic_training.head(20)
heuristic_training['heuristic'].value_counts()

                                       clean_sentence  order heuristic
0   yeah gr8 db8 ir8 st8 hey apreci8 gr8 wan mayb ...      0       NaN
1                       rel8 str8 appreci8 congratul8      1       NaN
2                                                 NaN      2       NaN
3                                        plz str8 ir8      3       NaN
4                                           cre8 cant      4       NaN
5               convers8 wont ber8 number 8888888 ask      5       NaN
6                                           calls st8      6       NaN
7                                            ask loc8      7       NaN
8     even full pl8 always time communic8 dont hesit8      8       NaN
9   pretty good tbh edit pretty good fuck sean lyi...      0       NaN
10                                  1st public review      0       NaN
11  edit quick tips left click enter menus right c...      1       NaN
12  press craft item inventory menu took like minu...      2       NaN
13  is

heuristic
[F]    498
[U]    497
[G]    490
Name: count, dtype: int64

In [339]:
heuristic_training.head(20)

,clean_sentence,order,heuristic
494,even tho kind added 219 support still stretche...,2,[F]
593,sit loading screen minutes crashes,0,[F]
599,games fantastic upon launching screen flashes ...,0,[F]
768,lets talk menus bad port obviously frustrating...,8,[U]
887,crashes loading space scene,0,[F]
892,terrible fps terrible graphic quality loading ...,0,[F]
979,stuttering freezing fixed things like searchin...,5,[F]
998,lowest fps ever stuttering everywhere,1,[F]
1165,issues launching game,3,[F]
1433,deafult 30fps unintuative menu system initally...,1,[U]


In [342]:
print(heuristic_training.head(10))

heuristic_training['heuristic_str'] = heuristic_training['heuristic']

heuristic_training['heuristic_str'] = [','.join(map(str, l)) for l in heuristic_training['heuristic']]

heuristic_training = (heuristic_training.groupby('heuristic_str', group_keys=False)
         .apply(lambda g: g.sample(n=min(300, len(g)), random_state=5))
      )

heuristic_training = heuristic_training.drop(columns='heuristic_str')

print(heuristic_training.head(10))

                                         clean_sentence  order heuristic
494   even tho kind added 219 support still stretche...      2       [F]
593                  sit loading screen minutes crashes      0       [F]
599   games fantastic upon launching screen flashes ...      0       [F]
768   lets talk menus bad port obviously frustrating...      8       [U]
887                         crashes loading space scene      0       [F]
892   terrible fps terrible graphic quality loading ...      0       [F]
979   stuttering freezing fixed things like searchin...      5       [F]
998               lowest fps ever stuttering everywhere      1       [F]
1165                              issues launching game      3       [F]
1433  deafult 30fps unintuative menu system initally...      1       [U]
                                           clean_sentence  order heuristic
119410        performance issues going really experienced      4       [F]
462017  extremely buggy base randomly deleted s

In [344]:
print(datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S"))

21.Mar 2025 00:22:09


In [346]:
print(len(heuristic_training))

900


In [349]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier

# Encode multi-labels
mlb = MultiLabelBinarizer()
label_matrix = mlb.fit_transform(heuristic_training['heuristic'])

# Convert text data into numerical features using TF-IDF
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(heuristic_training['clean_sentence'])



In [351]:
heuristic_training['heuristic'].value_counts()

heuristic
[F]    300
[G]    300
[U]    300
Name: count, dtype: int64

In [353]:
len(heuristic_training)

900

In [355]:
label_matrix

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       ...,
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]])

In [357]:
mlb.inverse_transform(np.array([[1, 1, 1]]))

[('F', 'G', 'U')]

In [359]:
mlb.classes_

array(['F', 'G', 'U'], dtype=object)

In [361]:
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
# Define cross-validation strategy (5-fold)
kf = KFold(n_splits=5, shuffle=True, random_state=1)

# Store F1 scores for each fold
f1_scores = []

for k in range(20, 60):
    # Perform cross-validation
    for fold, (train_index, test_index) in enumerate(kf.split(X_tfidf)):
        # Split data into training and testing sets
        X_train, X_test = X_tfidf[train_index], X_tfidf[test_index]
        y_train, y_test = label_matrix[train_index], label_matrix[test_index]
    
        # Train Binary Relevance model with KNN
        br_knn = OneVsRestClassifier(KNeighborsClassifier(n_neighbors=k))
        br_knn.fit(X_train, y_train)
    
        # Predict on test data
        y_pred = br_knn.predict(X_test)
    
        # Compute micro-average F1-score for multi-label classification
        f1 = f1_score(y_test, y_pred, average="micro", zero_division=0)
        f1_scores.append(f1)
    
        #print(f"k={k}: Fold {fold+1} - Micro F1-score: {f1:.4f}")

    # Compute average F1-score across all folds
    avg_f1_score = np.mean(f1_scores)
    print(f"k={k}: Average Micro F1-score across all folds: {avg_f1_score:.4f}")

k=20: Average Micro F1-score across all folds: 0.6983
k=21: Average Micro F1-score across all folds: 0.7124
k=22: Average Micro F1-score across all folds: 0.7039
k=23: Average Micro F1-score across all folds: 0.7099
k=24: Average Micro F1-score across all folds: 0.7066
k=25: Average Micro F1-score across all folds: 0.7099
k=26: Average Micro F1-score across all folds: 0.7062
k=27: Average Micro F1-score across all folds: 0.7089
k=28: Average Micro F1-score across all folds: 0.7072
k=29: Average Micro F1-score across all folds: 0.7085
k=30: Average Micro F1-score across all folds: 0.7067
k=31: Average Micro F1-score across all folds: 0.7076
k=32: Average Micro F1-score across all folds: 0.7064
k=33: Average Micro F1-score across all folds: 0.7071
k=34: Average Micro F1-score across all folds: 0.7055
k=35: Average Micro F1-score across all folds: 0.7056
k=36: Average Micro F1-score across all folds: 0.7040
k=37: Average Micro F1-score across all folds: 0.7039
k=38: Average Micro F1-score

In [398]:
#Train the model after identifying the best k=3
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf, label_matrix, test_size=0.15, random_state=1
)

#for k in range(21,26):
k = 21
# Train Binary Relevance model with KNN
br_knn = OneVsRestClassifier(KNeighborsClassifier(n_neighbors=k))
br_knn.fit(X_train, y_train)

# Predict on test data
y_pred = br_knn.predict(X_test)

# Evaluate model performance
print(f'\nK={k}')
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=mlb.classes_, zero_division=0))

# Convert predictions back to label format
y_pred_labels = mlb.inverse_transform(y_pred)
accuracy1 = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy1)
#print("\nPredicted Labels:\n", y_pred_labels)


K=21
Classification Report:
               precision    recall  f1-score   support

           F       1.00      0.76      0.86        45
           G       0.89      0.64      0.75        39
           U       0.91      0.57      0.70        51

   micro avg       0.94      0.65      0.77       135
   macro avg       0.93      0.66      0.77       135
weighted avg       0.93      0.65      0.77       135
 samples avg       0.65      0.65      0.65       135

Accuracy: 0.6518518518518519


In [400]:
informative_df.tail(10)

,clean_sentence,order,informative
523638,many ways customise experience,22,0.0
523639,like elite dangerous super easy baby mode lot ...,23,1.0
523640,game random gen presets,0,1.0
523641,voice acting,1,1.0
523642,gets old,2,1.0
523643,none less swell time,3,0.0
523644,game elements many games sewn one incredibly well,0,0.0
523645,bit survival fps space sim trading farming bas...,1,1.0
523646,result beautifully presented journey discovery...,2,0.0
523647,would recommend everyone adventurous,3,0.0



#Tidy up the label values
for row in wip_df.itertuples(index=True, name='Row'):
    
    tempVal = wip_df.loc[row.Index, 'heuristic']
    print(tempVal)
    if type(tempVal) is list:
        asd=0 #Do nothing
    elif type(tempVal) is float:
        wip_df.loc[row.Index, 'heuristic'] = []
    elif type(tempVal) is str:
        wip_df.loc[row.Index, 'heuristic'] = []
    elif len(tempVal)==0:
        wip_df.loc[row.Index, 'heuristic'] = []
    elif tempVal.isnull():
        wip_df.loc[row.Index, 'heuristic'] = []
    else:
        wip_df.loc[row.Index, 'heuristic'] = list(str(tempVal).replace(',','').replace("'",'').replace(' ',''))
    
    

In [403]:
print(datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S"))

21.Mar 2025 00:26:42


In [405]:
pd.options.mode.chained_assignment = None  # default='warn'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

print(len(wip_df['heuristic']))
for row in wip_df.itertuples(index=True, name='Row'):
    #print(row.Index)
    #print(wip_df['heuristic'][row.Index] )
    if type(wip_df['heuristic'][row.Index]) is str:
        tempVal = list(str(wip_df['heuristic'][row.Index]).replace(',','').replace("'",'').replace(' ',''))
        if len(tempVal)==0:
            asd = 0
            #wip_df['heuristic'][row.Index] = []
        else:
            #wip_df.loc[row.Index, 'heuristic'] = tempVal
            wip_df['heuristic'][row.Index] = tempVal
    #elif type(wip_df['heuristic'][row.Index]) is list:
        #Do nothing
        #asd=0
    #elif type(wip_df['heuristic'][row.Index]) is float:
        #Do nothing
        #asd=0
        #wip_df['heuristic'][row.Index] = []
    #elif wip_df['heuristic'][row.Index].isnull() or wip_df['heuristic'][row.Index] == '':
        #wip_df['heuristic'][row.Index] = []
    #else:
        #wip_df['heuristic'][row.Index] = []


wip_df['heuristic'].value_counts()

523648


heuristic
[F]          500
[U]          500
[G]          500
[G, U]        44
[F, U]        16
[G, F]        16
[F, G]         9
[U, F]         5
[U, G]         3
[F, G, U]      2
[G, F, U]      1
Name: count, dtype: int64

In [406]:
wip_df['informative'].value_counts()

informative
0.0    274687
1.0    248961
Name: count, dtype: int64

In [407]:
print(datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S"))

21.Mar 2025 00:26:46


In [408]:
backup = wip_df

In [418]:
print(datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S"))

21.Mar 2025 18:00:28


In [420]:


#for row in wip_df[(wip_df['informative'] == 1) & (wip_df['heuristic'].isnull()) ].itertuples(index=True, name='Row'):
for row in wip_df[(wip_df['informative'] == 1) ].itertuples(index=True, name='Row'):
    vect1 = vectorizer.transform([str(row.clean_sentence)])
    #print(wip_df[(wip_df['informative'] == 1) & (wip_df['heuristic'].isnull()) ].loc[row.Index, 'heuristic'])
    #print(br_knn.predict(vect1))
    #print(mlb.inverse_transform(br_knn.predict(vect1)))
    #print(row.Index)
    wip_df['heuristic'][row.Index] = mlb.inverse_transform(br_knn.predict(vect1))
    




In [422]:
print(datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S"))

21.Mar 2025 18:33:03


In [426]:
wip_df['heuristic'].value_counts()

heuristic
[()]      105738
[(G,)]     96809
[(F,)]     32608
[(U,)]     13806
Name: count, dtype: int64

In [428]:
wip_df['informative'].value_counts()

informative
0.0    274687
1.0    248961
Name: count, dtype: int64

In [430]:
mlb.classes_

array(['F', 'G', 'U'], dtype=object)

In [432]:
wip_df['heuristic'].value_counts()

heuristic
[()]      105738
[(G,)]     96809
[(F,)]     32608
[(U,)]     13806
Name: count, dtype: int64

In [434]:
# making boolean series for a team name
#g_filter = 'G' in  wip_df['heuristic']

tuple_df = wip_df[wip_df['heuristic'].apply(lambda x: isinstance(x, list))]

g_set = tuple_df[tuple_df['heuristic'].apply(lambda x: 'G' in x[0])]
u_set = tuple_df[tuple_df['heuristic'].apply(lambda x: 'U' in x[0])]
f_set = tuple_df[tuple_df['heuristic'].apply(lambda x: 'F' in x[0])]

#g_set['heuristic'].value_counts()

# filtering data
#g_set = wip_df.where(g_filter)



In [436]:
g_set['heuristic'].value_counts()

heuristic
[(G,)]    96809
Name: count, dtype: int64

In [463]:
g_set.tail(20)

,recommendationid,clean_sentence,tokenized_sentence,voted_up,order,informative,heuristic
523550,70409796,like flying around ship exploring,do you like just flying around in your ship an...,1,6,1.0,"[(G,)]"
523568,70410353,game keeps setting waypoint middle empty space...,the game keeps setting a waypoint in the middl...,1,6,1.0,"[(G,)]"
523584,70412310,scratched itches astroneer elite dangerous wel...,it scratched the itches that astroneer and eli...,1,3,1.0,"[(G,)]"
523586,70414573,everything,everything,1,0,1.0,"[(G,)]"
523591,70416173,space cool,space is cool,1,0,1.0,"[(G,)]"
523595,70416758,fun game exploring building must also add work...,fun game for exploring building i must also ad...,1,0,1.0,"[(G,)]"
523599,70418574,main story line utterly depressing uninstalled...,the main story line was so utterly depressing ...,0,0,1.0,"[(G,)]"
523600,70418574,main story ruined would otherwise wonderful ex...,the main story ruined what would have otherwis...,0,1,1.0,"[(G,)]"
523609,70425032,want commander fleet explore space exotic plan...,you want to be the commander of a fleet go for...,1,1,1.0,"[(G,)]"
523615,70425169,looking forward playing exploring,looking forward to playing and exploring more,1,4,1.0,"[(G,)]"


In [438]:
u_set['heuristic'].value_counts()

heuristic
[(U,)]    13806
Name: count, dtype: int64

In [440]:
f_set['heuristic'].value_counts()

heuristic
[(F,)]    32608
Name: count, dtype: int64

In [465]:
g_set.to_csv('./datasets/__sprint2_filtered_gameplay_review_set.csv')
f_set.to_csv('./datasets/__sprint2_filtered_functionality_review_set.csv')
u_set.to_csv('./datasets/__sprint2_filtered_usability_review_set.csv')

In [466]:
wip_df.head(20)

,recommendationid,clean_sentence,tokenized_sentence,voted_up,order,informative,heuristic
0,24845425,yeah gr8 db8 ir8 st8 hey apreci8 gr8 wan mayb ...,yeah m8 this is gr8 no db8 i r8 it an 8 i h8 2...,1,0,0.0,NaN
1,24845425,rel8 str8 appreci8 congratul8,i rel8 str8 appreci8 and congratul8,1,1,0.0,NaN
2,24845425,NaN,i r8 this b8 an 88,1,2,0.0,NaN
3,24845425,plz str8 ir8,plz no h8 im str8 ir8,1,3,0.0,NaN
4,24845425,cre8 cant,cre8 more cant w8,1,4,1.0,[()]
5,24845425,convers8 wont ber8 number 8888888 ask,we should convers8 i wont ber8 my number is 88...,1,5,0.0,NaN
6,24845425,calls st8,no calls l8 or out of st8,1,6,0.0,NaN
7,24845425,ask loc8,if on a d8 ask k8 to loc8,1,7,0.0,NaN
8,24845425,even full pl8 always time communic8 dont hesit8,even with a full pl8 i always have time to com...,1,8,1.0,[()]
9,24845428,pretty good tbh edit pretty good fuck sean lyi...,its pretty good tbh edit its pretty good but f...,0,0,0.0,NaN


In [446]:
#tuple_df.to_csv('./datasets/__sprint2_filtered_gameplay_review_setasdasdasdasd.csv')